# MVP de Engenharia de Dados

## Catarina Carneiro da Silva

Neste estudo o objetivo é analisar a influência de algumas características de um funcionário em seu salário, com foco no setor terciário (comércio, serviços e turismo). O dataset utilizado neste projeto são os microdados da RAIS 2022 (Relação Anual de Informações Sociais), proveniente do Ministério do Trabalho. Essa base de informações é muito abrangente, podendo originar diversas inferências. No entanto, para ter resultados mais objetivos com o desejado, utilizei um subconjunto da base original, considerando apenas as variáveis mais relevantes.

###Perguntas a serem respondidas:

1- A discriminação das mulheres pode ser comprovada por dados? Qual o salário médio das mulheres e dos homens?

2- A discriminação de raça pode ser comprovada por dados? Qual o salário médio de cada uma das raças analisadas?

3- Acredita-se que as pessoas mais novas costumam receber salários menores devido a menor experiencia, pode-se comprovar essa crença com dados?

4- Acredita-se que as pessoas com menor escolaridade costumam receber salários menores devido ao menor conhecimento, pode-se comprovar essa crença com dados?

5- São Paulo (código 35) é considerado um dos estados mais avançados do país e, por isso, com maior nivel de salários. Pode-se comprovar essa crença com dados?

6- Qual atividade do setor terciário apresenta o maior nível salarial?

### Carga - Camada Bronze

Fiz o download dos microdados da RAIS de 2022 no site do Ministério do Trabalho em um arquivo CSV e criei uma nova tabela no Databricks Comunity Edition com base nesse arquivo.

In [0]:
%sql
SELECT *
FROM rais2022

cnae,faixa_etaria,escolaridade,idade,cor,sexo,uf,remuneracao
94120,7,1,61,8,1,50,2014.0
84116,5,1,30,99,1,50,1116.52
84116,5,1,31,99,1,50,1091.7
84116,4,1,28,99,1,50,1091.7
84116,8,1,69,99,1,50,2075.62
84116,4,1,26,99,1,50,2553.11
84116,4,1,28,99,1,50,2533.8
84116,7,1,50,99,1,50,1212.0
84116,5,1,33,99,1,50,2724.79
84116,4,1,25,99,1,50,2706.22


###Catálogo de dados:

CNAE - CNAE da atividade do emprego segundo a classificação 2.0 com a segmentação por classe. Neste momento encontram-se todas as CNAES registradas na RAIS, no entanto para analisar somente o setor terciário, como pretendido, terei que filtrar apenas as classes entre 45111 e 69206 na próxima etapa.

Como são 673 classes registradas, deixarei para especificar cada uma na próxima etapa, quando os dados já estiverem filtrados para o setor desejado.

In [0]:
%sql
SELECT DISTINCT cnae
FROM rais2022
ORDER BY cnae DESC;

cnae
99008
97005
96092
96033
96025
96017
95291
95215
95126
95118


Faixa_etaria - Faixa de idade do funcionário, valores esperados:

1-entre 10 a 14 anos/

2-entre 15 a 17 anos/

3-entre 18 a 24 anos/

4-entre 25 a 29 anos/

5-entre 30 a 39 anos/

6-entre 40 a 49 anos/

7-entre 50 a 64 anos/

8-65 anos ou mais.

Na base de dados também encontrei a categoria 99, que são os dados informados como ignorados. Na próxima etapa precisarei filtrar essas informações.

In [0]:
%sql
SELECT DISTINCT faixa_etaria
FROM rais2022
ORDER BY faixa_etaria DESC;

faixa_etaria
99
8
7
6
5
4
3
2
1


Escolaridade - Escolaridade do funcionário, valores esperados:

1-Analfabeto

2-Até 5ª Incompleto

3-5ª Completo Fundamental

4-6ª a 9ª Fundamental

5-Fundamental Completo

6-Médio Incompleto

7-Médio Completo

8-Superior Incompleto

9-Superior Completo

10-Mestrado

11-Doutorado

Na base de dados também encontrei a categoria 99, que são os dados informados como ignorados. Na próxima etapa precisarei filtrar essas informações.

In [0]:
%sql
SELECT DISTINCT escolaridade
FROM rais2022
ORDER BY escolaridade DESC;

escolaridade
99
11
10
9
8
7
6
5
4
3


Idade - Idade do funcionário. Por conter valores próximos de 100 e idades abaixo dos 18 anos, essa série causa estranhamento e gera necessidade de atenção ao se trabalhar com esses dados.

In [0]:
%sql
SELECT DISTINCT idade
FROM rais2022
ORDER BY idade DESC;

idade
100
99
98
97
96
95
94
93
92
91


Cor - Raça/cor segundo funcionário, valores esperados:

1-Indígena

2- Branca

4- Preta

6- Amarela

8- Parda

9- Não informado

Na base de dados também encontrei a categoria 99, que são os dados informados como ignorados. Na próxima etapa precisarei filtrar essas informações assim como a categoria 9 (não informado).

In [0]:
%sql
SELECT DISTINCT cor
FROM rais2022
ORDER BY cor DESC;

cor
99
9
8
6
4
2
1


Sexo - Gênero segundo funcionário, valores esperados:

1- Masculino

2- Feminino

9- Ignorado

Na próxima etapa precisarei filtrar as informações da categoria 9, que são os dados informados como ignorados.

In [0]:
%sql
SELECT DISTINCT sexo
FROM rais2022
ORDER BY sexo DESC;

sexo
9
2
1


UF - 26 estados brasileiros e o Distrito Federal, valores esperados:
11 - Rondônia /	12 - Acre /	13 - Amazonas /	14 - Roraima /	15 - Para /	16 - Amapa /	17 - Tocantins /	21 - Maranhão /	22 - Piaui /	23 - Ceará /	24 - Rio Grande do Norte /	25 - Paraíba /	26 - Pernambuco /	27 - Alagoas /	28 - Sergipe /	29 - Bahia /	31 - Minas Gerais /	32 - Espírito Santo	/ 33 - Rio de Janeiro /	35 - São Paulo /	41 - Paraná /	42 - Santa Catarina /	43 - Rio Grande do Sul /	50 - Mato Grosso do Sul /	51 - Mato Grosso /	52 - Goiás /	53 - Distrito Federal

Na base de dados também encontrei a categoria 99, que são os dados informados como ignorados. Na próxima etapa precisarei filtrar essas informações.

In [0]:
%sql
SELECT DISTINCT uf
FROM rais2022
ORDER BY uf DESC;

uf
99
53
52
51
50
43
42
41
35
33


remuneração - Remuneração média dos funcionários

In [0]:
%sql
SELECT DISTINCT remuneracao
FROM rais2022
ORDER BY remuneracao DESC;

remuneracao
3755863.0
3700000.08
3568937.82
3543183.0
3520000.0
3510114.6
3500000.07
3480000.0
3474116.26
3381833.6


###Transformação - Camada Silver
Para facilitar a consulta e limpar a base de dados farei as transformações necessárias.
Primeiro deletarei a coluna idade, já que os dados parecem estranhos e na coluna faixa_etaria os valores já aparecem agrupados, o que facilita a análise e oferece maior credibilidade ao resultado.

Para trabalhar apenas o setor terciário, como pretendido, filtrarei a coluna cnae para conter apenas as CNAES desse setor (entre as classes 45111 e 69206).

Para limpar o dataset, excluirei os seguintes dados:
o gênero que não corresponde ao sexo Feminino ou Masculino,  faixa etária e escolaridade com resultados ignorados, a cor com informações diferentes das tradicionais e as uf ignoradas.

In [0]:
%sql

CREATE OR REPLACE TABLE rais2022_silver AS
SELECT cnae, escolaridade, faixa_etaria, cor, sexo, remuneracao, uf
FROM rais2022
WHERE cnae >= 45111 AND cnae <= 69206
  AND sexo!=9
  AND faixa_etaria !=99
  AND escolaridade !=99
  AND cor!=9
  AND cor!=99
  AND uf!=99;

num_affected_rows,num_inserted_rows


In [0]:
%sql
SELECT DISTINCT cnae
FROM rais2022_silver
ORDER BY cnae DESC;

cnae
69206
69125
69117
68226
68218
68102
66304
66291
66223
66215


A série de cnaes foi filtrada e, com dados mais limpos, acrescentei o dicionário, com a atividade ligada a cada classe. Nas séries online do Ministério do Trabalho fiz o download em formato CSV dos nomes das atividades relacionadas a cada classe. Criei uma nova tabela no Databricks Comunity Edition com essas informações.

In [0]:
%sql
SELECT *
FROM classes

Classe,Nome
69206,"Atividades de Contabilidade, Consultoria e Auditoria Contabil e Tributaria"
69125,Cartorios
69117,"Atividades Juridicas, Exceto Cartorios"
68226,Gestao e Administracao da Propriedade Imobiliaria
68218,"Intermediacao na Compra, Venda e Aluguel de Imoveis"
68102,Atividades Imobiliarias de Imoveis Proprios
66304,Atividades de Administracao de Fundos por Contrato ou Comissao
66291,"Atividades Auxiliares dos Seguros, da Previdencia Complementar e dos Planos de Saude nao Especificadas Anteriormente"
66223,"Corretores e Agentes de Seguros, de Planos de Previdencia Complementar e de Saude"
66215,Avaliacao de Riscos e Perdas



Seguem abaixo as novas séries, com as transformações realizadas conforme desejado.

In [0]:
%sql
SELECT DISTINCT sexo
FROM rais2022_silver
ORDER BY sexo DESC;

sexo
2
1


In [0]:
%sql
SELECT DISTINCT faixa_etaria
FROM rais2022_silver
ORDER BY faixa_etaria DESC;

faixa_etaria
8
7
6
5
4
3
2
1


In [0]:
%sql
SELECT DISTINCT escolaridade
FROM rais2022_silver
ORDER BY escolaridade DESC;

escolaridade
11
10
9
8
7
6
5
4
3
2


In [0]:
%sql
SELECT DISTINCT cor
FROM rais2022_silver
ORDER BY cor DESC;

cor
8
6
4
2
1


In [0]:
%sql
SELECT DISTINCT uf
FROM rais2022_silver
ORDER BY uf DESC;

uf
53
52
51
50
43
42
41
35
33
32


#####1- A discriminação das mulheres pode ser comprovada por dados? Qual o salário médio das mulheres e dos homens?

In [0]:
%sql
SELECT sexo, AVG(remuneracao) AS remuneracao_sexo
FROM rais2022_silver
GROUP BY sexo
ORDER BY remuneracao_sexo DESC;

sexo,remuneracao_sexo
1,4339.0324753202785
2,3482.809441164602


In [0]:
%sql
SELECT sexo, COUNT(*) AS genero
FROM rais2022_silver
GROUP BY sexo
ORDER BY genero DESC;

sexo,genero
1,7992716
2,6055556


O salário médio das mulheres do setor terciário foi de R$ 3.483 em 2022, o que representa 80% da remuneração média dos homens no setor e confirma o menor nível salarial do gênero feminino. Deve-se considerar que elas representam menos da metade (43,1%) dos trabalhadores do Brasil e é necessário pensar em formas de incentivar o aumento da entrada delas no mercado de trabalho.

#####2- A discriminação de raça pode ser comprovada por dados? Qual o salário médio de cada uma das raças analisadas?

In [0]:
%sql
SELECT cor, AVG(remuneracao) AS remuneracao_cor
FROM rais2022_silver
GROUP BY cor
ORDER BY remuneracao_cor DESC;

cor,remuneracao_cor
6,6769.0480832367
2,4775.8470701512815
1,3711.0810999702894
4,3035.58368649696
8,3019.519925042666


In [0]:
%sql
SELECT cor, COUNT(*) AS raca
FROM rais2022_silver
GROUP BY cor
ORDER BY raca DESC;

cor,raca
2,7334474
8,5630126
4,940329
6,116415
1,26928


A maior média salarial corresponde a quem se considera "6- amarelo" (R$ 6.769), seguida pela população "2- branca" (R$ 4.776) e as duas últimas categorias são das pessoas denominadas "4- pretas" (R$ 3.036) e "8- pardas" (R$ 3.020).

Apesar da população branca não ter as maiores remunerações, eles representam mais da metade (52,2%) dos trabalhadores brasileiros, confirmando a preferência do mercado de trabalho por esses funcionários. 

#####3- Acredita-se que as pessoas mais novas costumam receber salários menores devido a menor experiencia, pode-se comprovar essa crença com dados?

In [0]:
%sql
SELECT faixa_etaria, AVG(remuneracao) AS remuneracao_idade
FROM rais2022_silver
GROUP BY faixa_etaria
ORDER BY remuneracao_idade DESC;

faixa_etaria,remuneracao_idade
6,5198.012395385265
8,5170.564066844556
7,4910.34251181779
5,4441.567947496604
4,2961.2288557839524
3,1969.1549368812493
2,963.4823195818942
1,638.910453759723


In [0]:
%sql
SELECT faixa_etaria, COUNT(*) AS idade
FROM rais2022_silver
GROUP BY faixa_etaria
ORDER BY idade DESC;

faixa_etaria,idade
5,4066205
6,2997927
3,2461234
4,2309966
7,1918714
8,175003
2,116909
1,2314


Os colaboradores abaixo de 24 anos são os que recebem menores salários, confirmando a hipótese inicial da menor idade ser acompanhada por menor salário, enquanto aqueles entre 40 e 49 anos recebem em média os maiores valores.

Pode-se observar um preferência do mercado de trabalho por contratar pessoas entre 30 e 49 anos, somando esses dois grupos encontra-se 50,3% dos funcionários brasileiros. Sendo funcionário mais experientes, no entanto não tão velhos para se tornarem improdutivos.

#####4- Acredita-se que as pessoas com menor escolaridade costumam receber salários menores devido ao menor conhecimento, pode-se comprovar essa crença com dados?

In [0]:
%sql
SELECT escolaridade, AVG(remuneracao) AS remuneracao_escolaridade
FROM rais2022_silver
GROUP BY escolaridade
ORDER BY remuneracao_escolaridade DESC;

escolaridade,remuneracao_escolaridade
10,26114.70055458715
11,18217.16102560787
9,11494.694465088623
8,4576.643150751637
7,2489.364135697803
5,2152.43010877633
4,2098.56872691009
1,2094.6398189387737
3,2047.5979745397283
6,2043.379161377498


In [0]:
%sql
SELECT escolaridade, COUNT(*) AS formacao
FROM rais2022_silver
GROUP BY escolaridade
ORDER BY formacao DESC;

escolaridade,formacao
7,8335590
9,2160002
6,992437
5,940002
8,792522
4,458750
3,163706
2,143959
10,28111
1,25461


Os dados comprovam que há uma grande diferença salarial dependendo do grau de estudo do funcionário, com as pessoas com escolaridade até o 5º ano incompleto recebendo os menores salários (R$ 1.998). É interessante observar que o nível de renda de quem possui mestrado (R$ 26.115) é acima de quem tem doutorado (R$ 18.217), provalvemente por terem menos opções de carreira especificamente voltada para os doutorando.

Em relação a quantidade de pessoas empregadas, a maior parte possui o ensino médio completo (59,3%), seguida pela população com o nível superior completo (15,4%).

#####5- São Paulo (código 35) é considerado um dos estados mais avançados do país e, por isso, com maior nivel de salários. Pode-se comprovar essa crença com dados?

In [0]:
%sql
SELECT uf, AVG(remuneracao) AS remuneracao_uf
FROM rais2022_silver
GROUP BY uf
ORDER BY remuneracao_uf DESC;

uf,remuneracao_uf
53,7151.892257216753
35,4815.875911582386
43,4088.623444486743
33,4015.8067896248176
42,3901.0926059527487
41,3767.8149652978695
51,3594.150289306611
26,3274.479465692664
31,3208.2392055226446
52,3190.5695646853187


In [0]:
%sql
SELECT uf, COUNT(*) AS estado
FROM rais2022_silver
GROUP BY uf
ORDER BY estado DESC;

uf,estado
35,4560038
31,1382166
33,1323783
41,897924
43,859083
42,618979
29,547159
52,420845
23,415390
26,410045


Os dados mostram que a localidade com maior nível salarial é o Distrito Federal (R$ 7.152), retirando o protagonismo do sudeste, no entanto, compatível por ser uma área política, ficando São Paulo (R$ 4.816) em segundo lugar. Apesar de refutarmos a hipótese desse estado ter o maior rendimento, São Paulo compreende a maior parte dos trabalhadores brasileiros (32,5%).

#####6- Qual atividade do setor terciário apresenta o maior nível salarial?

In [0]:
%sql
SELECT cnae, AVG(remuneracao) AS remuneracao_cnae
FROM rais2022_silver
GROUP BY cnae
ORDER BY remuneracao_cnae DESC;

cnae,remuneracao_cnae
64239,44606.74423299517
64336,35301.324690454385
64328,26944.89764044943
64611,24730.25076690215
64344,21592.638803582846
51307,21231.015
65308,19520.698817204302
64999,18691.44293066313
49400,18576.17177103448
64212,18203.867016377895


In [0]:
%sql
SELECT cnae, COUNT(*) AS classe
FROM rais2022_silver
GROUP BY cnae
ORDER BY classe DESC;

cnae,classe
47113,1245976
56112,976229
49302,890320
47440,512026
47814,505605
47717,470442
45307,372989
49213,339547
47890,327442
47121,318698


O maior nível salarial encontrado foi nas atividades de serviços financeiros, sendo as quatro primeiras ligadas a essas funções: Caixas Econômicas - R$ 44.607 / Bancos de Desenvolvimento - R$ 35.301 / Bancos de Investimento - R$ 26.945 / Holdings de Instituições Financeiras - R$ 24.730.

A maior parte do quadro de funcionários (15,8%) trabalha em atividades alimentícias. Sendo que 8,9%	dos trabalhadores brasileiros no setor terciário estão concetrados no Comércio Varejista de Mercadorias em Geral, com Predominância de Produtos Alimentícios - Hipermercados e Supermercados (Cnae 47113) e 6,9%	trabalham em Restaurantes e Outros Estabelecimentos de Serviços de Alimentação e Bebidas (Cnae 56112).




###Autoavaliação

Com esse estudo consegui observar as principais características do mercado de trabalho brasileiro, sendo possível responder as perguntas propostas. Para isso não precisei chegar até a camada Gold, com a camada Silver sendo o suficiente para atingir o objetivo. Também não foi necessário criar um modelo de dados em Esquema Estrela ou Snowflake, pois a tabela flat conseguiu responder as perguntas pretendidas no estudo.

In [0]:
%sql
SELECT 
    (SELECT CORR(sexo, remuneracao) FROM rais2022_silver) AS correlation_sexo,
    (SELECT CORR(cor, remuneracao) FROM rais2022_silver) AS correlation_cor,
    (SELECT CORR(faixa_etaria, remuneracao) FROM rais2022_silver) AS correlation_idade,
    (SELECT CORR(escolaridade, remuneracao) FROM rais2022_silver) AS correlation_escolaridade,
    (SELECT CORR(uf, remuneracao) FROM rais2022_silver) AS correlation_uf;

correlation_sexo,correlation_cor,correlation_idade,correlation_escolaridade,correlation_uf
-0.027103351053910783,-0.05164146138749974,0.07156180731501754,0.15498704188918808,0.026432627425250318


Pela tabela de correlação entre os dados podemos perceber uma correlação negativa entre gênero e salário, mostrando que o fato de ser mulher (indicador 2) afeta negativamente a remuneração. Em relação a raça, o indicador também ficou abaixo de 0, confirmando que os indicadores maiores (parda, amarela, preta) tem influência negativa. No entanto, ambas as correlação são próximas a 0, representando que o impacto não é tão significativo quanto para as outras características.

Tanto a faixa etária, quando o estado de origem e a escolaridade apresentaram correlações positivas, sendo a da escolaridade a mais significativa. Por isso, podemos inferir que a escolaridade é a caraterística que apresenta maior impacto no nível salarial do trabalhador.

A correlação com as Cnaes não foi realizada pois elas não possuem uma ordem crescente lógica para analisar, sendo a influência de se ter um número de classe maior ou menos uma informação sem relevância.

Uma evolução desse estudo seria analisar as mesmas informações na segmentação de estado, para observar como cada estado se comporta em relação a contratação de mulheres, negros, pessoas mais velhas, com menor escolaridade ou quaisquer outras considerações que julgue relevantes.
Também há a possibilidade de analisar essas características dentro de uma cnae específica.